### Introduction: This is the Jupyter notebook to do the flowing things:

1. Read slimmed PKU Tree files
2. Store the raw MC histograms to pickle files

kernel:HWW



### Import necessary modules

In [38]:
import numpy as np
import os
import pandas as pd
import random
import awkward as ak
import matplotlib as mpl
import matplotlib.pyplot as plt
import mplhep as hep
import boost_histogram as bh
from scipy import interpolate
# from sklearn.metrics import roc_curve, auc
from cycler import cycler
import uproot
# means uproot4
import ROOT
import argparse
import glob
import json
import logging
import os
import pickle as pkl
import warnings

import hist as hist2
import pyarrow
# import utils #local file: utils.py
import yaml
from typing import Dict, List, Union
from dataclasses import dataclass, field
from copy import deepcopy

# from coffea import hist
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.nanoevents.methods import vector
# from coffea.nanoevents.methods.vector import PtEtaPhiMLorentzVector

### Read SlimmedTree files

In [39]:
#load the slimmedtree files using uproot

#different year available here.
year = "2016APV"
# year = "2016"
# year = "2017"
# year = "2018"
# year = "Full-Run2"

#if run on PKU cluster, use this:
# CustNanoData = {
#     'data'        : "/data/bond/zhaoyz/SlimmedTree/V5/%s/Data/SlimmedTree_Data.root"%(year),
#     'QCD'         : "/data/bond/zhaoyz/SlimmedTree/V5/%s/MC/SlimmedTree_QCD.root"%(year),
#     'Top'         : "/data/bond/zhaoyz/SlimmedTree/V5/%s/MC/SlimmedTree_Top.root"%(year),
#     'WJets'       : "/data/bond/zhaoyz/SlimmedTree/V5/%s/MC/SlimmedTree_WJets.root"%(year),
#     'Rest'        : "/data/bond/zhaoyz/SlimmedTree/V5/%s/MC/SlimmedTree_Rest.root"%(year),
#     'TotalSignal' : "/data/bond/zhaoyz/SlimmedTree/V5/%s/Signal/SlimmedTree_Total.root"%(year),
#     'ggF'         : "/data/bond/zhaoyz/SlimmedTree/V5/%s/Signal/SlimmedTree_GluGlu.root"%(year),
#     'VH'          : "/data/bond/zhaoyz/SlimmedTree/V5/%s/Signal/SlimmedTree_VH.root"%(year),
#     'ttH'         : "/data/bond/zhaoyz/SlimmedTree/V5/%s/Signal/SlimmedTree_ttH.root"%(year),
#     'VBF'         : "/data/bond/zhaoyz/SlimmedTree/V5/%s/Signal/SlimmedTree_VBF.root"%(year),
# } 

#if run on lxplus, use this:
# CustNanoData = {
#     'data'        : "/eos/user/y/yuzhe/HWW/SlimmedTree/V5/%s/Data/SlimmedTree_Data.root"%(year),
#     'QCD'         : "/eos/user/y/yuzhe/HWW/SlimmedTree/V5/%s/MC/SlimmedTree_QCD.root"%(year),
#     'Top'         : "/eos/user/y/yuzhe/HWW/SlimmedTree/V5/%s/MC/SlimmedTree_Top.root"%(year),
#     'WJets'       : "/eos/user/y/yuzhe/HWW/SlimmedTree/V5/%s/MC/SlimmedTree_WJets.root"%(year),
#     'Rest'        : "/eos/user/y/yuzhe/HWW/SlimmedTree/V5/%s/MC/SlimmedTree_Rest.root"%(year),
#     'TotalSignal' : "/eos/user/y/yuzhe/HWW/SlimmedTree/V5/%s/Signal/SlimmedTree_Total.root"%(year),
#     'ggF'         : "/eos/user/y/yuzhe/HWW/SlimmedTree/V5/%s/Signal/SlimmedTree_GluGlu.root"%(year),
#     'VH'          : "/eos/user/y/yuzhe/HWW/SlimmedTree/V5/%s/Signal/SlimmedTree_VH.root"%(year),
#     'ttH'         : "/eos/user/y/yuzhe/HWW/SlimmedTree/V5/%s/Signal/SlimmedTree_ttH.root"%(year),
#     'VBF'         : "/eos/user/y/yuzhe/HWW/SlimmedTree/V5/%s/Signal/SlimmedTree_VBF.root"%(year),
# }  

#if run on CMSconnect, use this:
CustNanoData = {
    'data'        : "/ospool/cms-user/yuzhe/SlimmedTree/V6/%s/Data/SlimmedTree_Data.root"%(year),
    'QCD'         : "/ospool/cms-user/yuzhe/SlimmedTree/V6/%s/MC/SlimmedTree_QCD.root"%(year),
    'TT'          : "/ospool/cms-user/yuzhe/SlimmedTree/V6/%s/MC/SlimmedTree_TT.root"%(year),
    'ST'          : "/ospool/cms-user/yuzhe/SlimmedTree/V6/%s/MC/SlimmedTree_ST.root"%(year),
    'WJets'       : "/ospool/cms-user/yuzhe/SlimmedTree/V6/%s/MC/SlimmedTree_WJets.root"%(year),
    'Rest'        : "/ospool/cms-user/yuzhe/SlimmedTree/V6/%s/MC/SlimmedTree_Rest.root"%(year),
    'TotalSignal' : "/ospool/cms-user/yuzhe/SlimmedTree/V6/%s/Signal/SlimmedTree_Total.root"%(year),
    'ggF'         : "/ospool/cms-user/yuzhe/SlimmedTree/V6/%s/Signal/SlimmedTree_GluGlu.root"%(year),
    'ZH'          : "/ospool/cms-user/yuzhe/SlimmedTree/V6/%s/Signal/SlimmedTree_WH.root"%(year),
    'WH'          : "/ospool/cms-user/yuzhe/SlimmedTree/V6/%s/Signal/SlimmedTree_HZJ.root"%(year),
    'ttH'         : "/ospool/cms-user/yuzhe/SlimmedTree/V6/%s/Signal/SlimmedTree_ttH.root"%(year),
    'VBF'         : "/ospool/cms-user/yuzhe/SlimmedTree/V6/%s/Signal/SlimmedTree_VBF.root"%(year),
}        


BKG = ["QCD","Top","WJets","Rest"]
files = {typefile : {} for typefile in CustNanoData}
for typefile in CustNanoData:
    files[typefile] = uproot.lazy({CustNanoData[typefile]: "PKUTree" })
    
#for signal decomposition plots, we store these information in different objects
files[r"$H^{4q}$"] = files["TotalSignal"][files["TotalSignal"]["R4q_a"] == 1]
files[r"$H^{3q}$"] = files["TotalSignal"][(files["TotalSignal"]["R3q_a"] == 1) & (files["TotalSignal"]["R3q_taudecay_a"] != 1)]
files["W"] = files["TotalSignal"][files["TotalSignal"]["w_a"] == 1]
files["top"] = files["TotalSignal"][files["TotalSignal"]["t_a"] == 1]
files["Z"] = files["TotalSignal"][files["TotalSignal"]["z_a"] == 1]
files[r"$H^{lqq}$"] = files["TotalSignal"][(files["TotalSignal"]["Rlqq_a"] == 1) | (files["TotalSignal"]["R3q_taudecay_a"] == 1)]
files["g/q"] = files["TotalSignal"][files["TotalSignal"]["gKK_g_a"] == 1]
files["rest"] = files["TotalSignal"][(files["TotalSignal"]["u_a"] == 1) | (files["TotalSignal"]["Rlq_a"] == 1) | (files["TotalSignal"]["R2q_a"] == 1)]


### Get DPhi in the events

In [40]:
def get_dphi(events):
    pT_higgs   = events["PTj_V2_a"]
    eta_higgs  = events["Etaj_V2_a"]
    phi_higgs  = events["Phij_V2_a"]
    mass_higgs = events["Mj_V2_a"]
    pT_MET = events["MET_et"]
    eta_MET = events["Etaj_V2_a"]
    phi_MET = events["MET_phi"]
    mass_MET = ak.zeros_like(events["MET_phi"])
    vec_higgs = ak.zip({
        "pt"   : pT_higgs   ,
        "eta"  : eta_higgs  ,
        "phi"  : phi_higgs  ,
        "mass" : mass_higgs ,
    },
    with_name="PtEtaPhiMLorentzVector",
    behavior=vector.behavior,
    )
    vec_MET = ak.zip({
        "pt"   :   pT_MET   ,
        "eta"  :  eta_MET  ,
        "phi"  :  phi_MET  ,
        "mass" : mass_MET ,
    },
    with_name="PtEtaPhiMLorentzVector",
    behavior=vector.behavior,
    )
    delta_phi = np.subtract(vec_MET.phi, vec_higgs.phi)
    delta_phi = np.where(delta_phi > np.pi, delta_phi - 2*np.pi, delta_phi)
    delta_phi = np.where(delta_phi < -np.pi, delta_phi + 2*np.pi, delta_phi)
    delta_phi = np.abs(delta_phi)
    print(delta_phi)
    events["DPhi"] = delta_phi

In [41]:
for k in files:
    print("Add dphi of:",k)
    get_dphi(events=files[k])

Add dphi of: data


[3.14, 0.298, 2.27, 0.915, 0.409, 0.824, ... 1.54, 0.654, 0.7, 2.53, 1.87, 0.291]
Add dphi of: QCD
[2.31, 3.08, 2.34, 3.11, 2.24, 2.81, ... 0.702, 0.273, 0.0464, 2.74, 3.07, 0.518]
Add dphi of: TT
[3.04, 0.473, 3.05, 1.31, 0.713, 0.0965, ... 0.407, 0.131, 3.02, 2.47, 2.96, 1.77]
Add dphi of: ST
[2.41, 0.636, 1.02, 0.0351, 2.12, 0.928, 1.44, ... 1.48, 3.02, 2.9, 2.58, 2.77, 0.61]
Add dphi of: WJets
[0.108, 0.158, 2, 0.181, 0.337, 2.65, 0.613, ... 2.6, 2.94, 0.249, 0.245, 2.88, 2.77]
Add dphi of: Rest
[2.99, 0.122, 3.07, 0.258, 0.213, 2.38, ... 1.85, 1.57, 0.00223, 0.102, 0.152, 0.309]
Add dphi of: TotalSignal
[1.89, 0.0198, 0.206, 3.14, 0.668, 1.61, ... 0.424, 0.474, 3.13, 0.239, 0.263, 0.227]
Add dphi of: ggF
[1.89, 0.0198, 0.206, 3.14, 0.668, 1.61, ... 0.0329, 0.688, 0.31, 0.00507, 2.89, 2.9]
Add dphi of: ZH
[0.102, 3.03, 0.153, 2.24, 2.7, 3, ... 0.424, 0.474, 3.13, 0.239, 0.263, 0.227]
Add dphi of: WH
[0.424, 2.3, 1.97, 1.23, 2.17, 2.47, ... 0.084, 1.92, 0.409, 1.61, 2.1, 0.0788]
Add

### Get MET recovery mass

In [42]:

def get_reco(events,mass = "Mj_corr_V2_a", MET_UE = None):
    pT_higgs   = events["PTj_V2_a"]
    eta_higgs  = events["Etaj_V2_a"]
    phi_higgs  = events["Phij_V2_a"]
    mass_higgs = events[mass]
    if not MET_UE:
        pT_MET = events["MET_et"]
        phi_MET = events["MET_phi"]
    elif MET_UE == "up":
        pT_MET = events["MET_et_UEup"]
        phi_MET = events["MET_phi_UEup"]
    elif MET_UE == "down":
        pT_MET = events["MET_et_UEdown"]
        phi_MET = events["MET_phi_UEdown"]
    eta_MET = events["Etaj_V2_a"]
    mass_MET = ak.zeros_like(events["MET_phi"])
    vec_higgs = ak.zip({
        "pt"   : pT_higgs   ,
        "eta"  : eta_higgs  ,
        "phi"  : phi_higgs  ,
        "mass" : mass_higgs ,
    },
    with_name="PtEtaPhiMLorentzVector",
    behavior=vector.behavior,
    )

    vec_MET = ak.zip({
        "pt"   :   pT_MET   ,
        "eta"  :  eta_MET  ,
        "phi"  :  phi_MET  ,
        "mass" : mass_MET ,
    },
    with_name="PtEtaPhiMLorentzVector",
    behavior=vector.behavior,
    )

    vec_sum = vec_MET + vec_higgs
    mH_reco = vec_sum.mass
    if not MET_UE: mass_str = "MH_Reco" if mass == "Mj_corr_V2_a" else mass
    else : mass_str = "MH_Reco" + "_UE_" + MET_UE
    
    if not MET_UE: 
        events[mass_str] = ak.where(( (events["DPhi"] < 0.8) & (events["MET_et"]/events["PTj_V2_a"] > 0.1)), mH_reco, events[mass])
    elif MET_UE == "up" : 
        events[mass_str] = ak.where(( (events["DPhi"] < 0.8) & (events["MET_et_UEup"]/events["PTj_V2_a"] > 0.1)), mH_reco, events[mass])
    elif MET_UE == "down" : 
        events[mass_str] = ak.where(( (events["DPhi"] < 0.8) & (events["MET_et_UEdown"]/events["PTj_V2_a"] > 0.1)), mH_reco, events[mass])

In [43]:
for k in files:
    # note that QCD and Data don't have such variation
    if k == "data" or k == "QCD": continue
    print("Add reco of:",k)
    get_reco(events=files[k])
    get_reco(events=files[k],MET_UE = "up")
    get_reco(events=files[k],MET_UE = "down")
    
    get_reco(events=files[k],mass = "Mj_jesTotalUp_a")
    get_reco(events=files[k],mass = "Mj_jesTotalDown_a")
    get_reco(events=files[k],mass = "Mj_jerUp_a")
    get_reco(events=files[k],mass = "Mj_jerDown_a")
    
    get_reco(events=files[k],mass = "Mj_jmsUp_a")
    get_reco(events=files[k],mass = "Mj_jmsDown_a")
    get_reco(events=files[k],mass = "Mj_jmrUp_a")
    get_reco(events=files[k],mass = "Mj_jmrDown_a")

Add reco of: TT


Add reco of: ST
Add reco of: WJets
Add reco of: Rest
Add reco of: TotalSignal
Add reco of: ggF
Add reco of: ZH
Add reco of: WH
Add reco of: ttH
Add reco of: VBF
Add reco of: $H^{4q}$
Add reco of: $H^{3q}$
Add reco of: W
Add reco of: top
Add reco of: Z
Add reco of: $H^{lqq}$
Add reco of: g/q
Add reco of: rest


### some test about variables / output all the variables

In [44]:
files["TotalSignal"].fields

['a_TopbWqq1c',
 'a_HWqqWq2c',
 'nb_m_deep_ex',
 'a_TopbWmv',
 'nb_l_deep_in',
 'puWeightDown',
 'Rlqq_a',
 'b_HWWvsQCD_V2',
 'LHEScaleWeight_1',
 'a_HWqqWq1c',
 'a_QCDc',
 'Mj_V2_c',
 'R2q_a',
 'R3q_taudecay_c',
 'puWeightUp',
 'Mj_jmrDown_b',
 'R4q_b',
 'a_QCDbb',
 'gKK_g_b',
 'a_TopbWq0c',
 'a_Hcc',
 'tlqq_a',
 'Mj_jerUp_b',
 'Mj_corr_3',
 'Etaj',
 'PTj_2',
 'PTj_mid',
 'a_HWqqWtauhv0c',
 'w_b',
 'LHEScaleWeight_6',
 'LHEScaleWeight_2',
 'Mj',
 'Rlq_a',
 'tlqq_b',
 'a_HWqqWtaumv1c',
 'Mj_jmsUp_b',
 'MET_et_UEup',
 'tlqq_c',
 'Mj_3',
 'a_Htauhtauh',
 'a_HWqqWev0c',
 'u_a',
 'a_HWqqWqq2c',
 'a_HWqqWqq1c',
 'Mj_jmsDown_c',
 'z_a',
 'R3q_taudecay_b',
 'puWeight',
 'Etaj_V2_c',
 'c_HWWvsQCD_V2',
 'Mj_jerDown_b',
 'R2q_c',
 'Mj_jmrUp_b',
 'a_QCDb',
 'Rlqq_c',
 'PSWeight_2',
 'MET_T1Smear_phi_XYcorr',
 'a_HWqqWqq0c',
 'a_QCDothers',
 'Mj_jesTotalUp_c',
 'MJJ',
 'a_HWqqWtauhv1c',
 'b_HWW_V2',
 'Mj_jerDown_c',
 'Mj_jmsDown_b',
 'Mj_jesTotalUp_b',
 'nb_m_deep_in',
 'PSWeight_1',
 'Mj_V2_b',
 

### plot setting

In [45]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import mplhep as hep
import boost_histogram as bh
from cycler import cycler

use_helvet = False ## true: use helvetica for plots, make sure the system have the font installed
if use_helvet:
    CMShelvet = hep.style.CMS
    CMShelvet['font.sans-serif'] = ['Helvetica', 'Arial']
    plt.style.use(CMShelvet)
else:
    plt.style.use(hep.style.CMS)

def flow(hist: bh.Histogram, overflow: bool=True, underflow: bool=True):
    h, var = hist.view(flow=(overflow | underflow)).value, hist.view(flow=(overflow | underflow)).variance
    if overflow: 
        # h, var also include underflow bins but in plots usually no underflow data
        # And we've filled None with -999, so we shouldn't show underflow data (mostly from filled None)
        # You have to access the overflow and underflow bins data like below:
        h[-2] += h[-1]; var[-2] += var[-1]
    if underflow:
        h[1] += h[0]; var[1] += var[0]
    if overflow or underflow:
        h, var = h[1:-1], var[1:-1]
    return h, var
    # Return the updated histogram and variance

def error_bar(h, var, type='data'):
    from scipy.interpolate import CubicSpline
    if type == 'data':
        number = h
    elif type == 'mc':  # h = k*N, var = k^2*N, std = k*sqrt(N)
        number = h**2 / var
    else:
        raise ValueError("type should be 'data' or 'mc'! ")
    center = range(11) # Number: 0-10
    up = np.array([1.84, 3.30, 4.64, 5.92, 7.16, 8.38, 9.58, 10.77, 11.95, 13.11, 14.27]) - center
    down = center - np.array([0, 0.17, 0.71, 1.37, 2.09, 2.84, 3.62, 4.42, 5.23, 6.06, 6.89])
    #cs means to create a CubicSpline object
    cs_up = CubicSpline(x=center, y=up)
    cs_down = CubicSpline(x=center, y=down)
    
    Garwood = (number>0)&(number<10)
    poison_error_bar = np.sqrt(number)
    up_error_bar = np.copy(poison_error_bar)
    down_error_bar = np.copy(poison_error_bar)
    up_error_bar[Garwood] = cs_up(number[Garwood])
    down_error_bar[Garwood] = cs_down(number[Garwood])
    if type == 'mc':
        up_error_bar *= var/h
        down_error_bar *= var/h
    up_error_bar [up_error_bar < 0 ] = 0
    down_error_bar [down_error_bar < 0 ] = 0
    return np.array([down_error_bar, up_error_bar])


# function to find the optimal region with S/sqrt(B)
# not used so far
def optimalcut(shist, bhist):
    n_bins = len(shist)
    best_lower = None
    best_upper = None
    best_s_sqrt_b = 0

    for lower in range(n_bins):
        for upper in range(lower+1, n_bins+1):
            s = np.sum(shist[lower:upper])
            b = np.sum(bhist[lower:upper])
            s_sqrt_b = s / np.sqrt(b + 1)

            if s_sqrt_b > best_s_sqrt_b:
                best_lower = lower
                best_upper = upper
                best_s_sqrt_b = s_sqrt_b

    return best_lower, best_upper, best_s_sqrt_b

def optimalcut_oneside(shist, bhist, epsilon = 0.01):
    '''
    Given the signal histogram and background histogram, 
    show the one-side cut for the variable to get best s/sqrt(b).
    Args:
        shist:signal histogram
        bhist:background histogram
        epsilon(float): epsilon to avoid numerical errs 
    '''
    n_bins = len(shist)
    best_cut = 0
    best_s_sqrt_b = 0

    for cut in range(n_bins):
        s = np.sum(shist[cut:])
        b = np.sum(bhist[cut:])
        s_sqrt_b = s / np.sqrt(b + epsilon)
        if s_sqrt_b > best_s_sqrt_b:
            best_cut = cut
            best_s_sqrt_b = s_sqrt_b
        
    return best_cut, best_s_sqrt_b

def optimalcut_mid_combine(shist1, shist2, bhist, epsilon = 1):
    '''
    Given the signal histogram and background histogram, 
    show the one-side cut for the variable to get best s/sqrt(b).
    Args:
        shist:signal histogram
        bhist:background histogram
        epsilon(float): epsilon to avoid numerical errs 
    '''
    n_bins = len(shist1)
    best_cut = 0
    best_combined_sig_two_side = 0

    for cut in range(n_bins):
        s_right_side = np.sum(shist2[cut:])
        b_right_side = np.sum(bhist[cut:])
        s_left_side = np.sum(shist1[:cut])
        b_left_side = np.sum(bhist[:cut])
        s_sqrt_b_right_side = s_right_side / np.sqrt(b_right_side + epsilon)
        s_sqrt_b_left_side = s_left_side / np.sqrt(b_left_side + epsilon)
        combined_sig_two_side = np.sqrt((s_sqrt_b_right_side)**2 + (s_sqrt_b_left_side)**2)
        if combined_sig_two_side > best_combined_sig_two_side:
            best_cut = cut
            best_combined_sig_two_side = combined_sig_two_side
        
    return best_cut, best_combined_sig_two_side


### Define templates dir

In [46]:
MAIN_DIR = "."

plot_dir = f"{MAIN_DIR}/templates/18Mar24"
_ = os.system(f"mkdir -p {plot_dir}")

### Define observable object variables

In [47]:
@dataclass
class ShapeVar:
    """Class to store attributes of a variable to make a histogram of.

    Args:
        var (str): variable name
        label (str): variable label
        bins (List[int]): bins
        reg (bool, optional): Use a regular axis or variable binning. Defaults to True.
        blind_window (List[int], optional): if blinding, set min and max values to set 0. Defaults to None.
        significance_dir (str, optional): if plotting significance, which direction to plot it in.
          See more in plotting.py:ratioHistPlot(). Options are ["left", "right", "bin"]. Defaults to "right".
    """

    var: str = None
    label: str = None
    bins: List[int] = None
    reg: bool = True #regular axis
    blind_window: List[int] = None
    significance_dir: str = "right"

    def __post_init__(self):
        # create axis used for histogramming
        if self.reg:
            self.axis = hist2.axis.Regular(*self.bins, name=self.var, label=self.label)
        else:
            self.axis = hist2.axis.Variable(self.bins, name=self.var, label=self.label)

@dataclass
class Syst:
    samples: list[str] = None
    years: list[str] = field(default_factory=lambda: years)
    label: str = None
    
def blindBins(h: hist2.Hist, blind_region: List, blind_sample: str = None, axis=0):
    """
    Blind (i.e. zero) bins in histogram ``h``.
    If ``blind_sample`` specified, only blind that sample, else blinds all.
    """
    if axis > 0:
        raise Exception("not implemented > 1D blinding yet")

    bins = h.axes[axis + 1].edges
    lv = int(np.searchsorted(bins, blind_region[0], "right"))
    rv = int(np.searchsorted(bins, blind_region[1], "left") + 1)

    if blind_sample is not None:
        data_key_index = np.where(np.array(list(h.axes[0])) == blind_sample)[0][0]
        h.view(flow=True)[data_key_index][lv:rv].value = 0
        h.view(flow=True)[data_key_index][lv:rv].variance = 0
    else:
        h.view(flow=True)[:, lv:rv].value = 0
        h.view(flow=True)[:, lv:rv].variance = 0       
shape_vars = [
    ShapeVar(
        "MH_Reco",
        r"Higgs candidate MET recovery mass [GeV]",
        [20, 50, 250],
        reg=True,
        blind_window=[90, 150],
    )
]


### Define samples we consider

In [48]:
sig_keys = [
    "ggF",
    "VBF",
    "ZH",
    "WH",
    "ttH",
]

bkg_keys = [
    "TT",
    "WJets",
    "ST",
    "Rest"
]

mc_keys = sig_keys + bkg_keys

### Define weight shift list

In [50]:
years = ["2016APV", "2016", "2017", "2018"]

weight_shifts = {
    "pileup": Syst(samples=mc_keys, label="Pileup"),
    "ISRPartonShower": Syst(samples=mc_keys, label="ISR Parton Shower"),
    "FSRPartonShower": Syst(samples=mc_keys, label="FSR Parton Shower"),
    "QCDscale": Syst(samples=bkg_keys, label="QCDScale"),
    "trigger" : Syst(samples=mc_keys, label="Trigger SF"),
}

### Define variation shift list

In [51]:
jecs = {
    "JES": "JES_jes",
    "JER": "JER",
}

uncluste = {
    "UE": "unclusteredEnergy",
}

jec_shifts = []
for key in jecs:
    for shift in ["up", "down"]:
        jec_shifts.append(f"{key}_{shift}")

ue_shifts = []
for key in ue_shifts:
    for shift in ["up", "down"]:
        ue_shifts.append(f"{key}_{shift}")

### Define CUT(aka. regions)

In [9]:
CUT = {        
    "SR1a" : {k:  (files[k]["nb_t_deep_in"] == 0 ) & (files[k]["MH_Reco"] >= 50) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] <= 0.25) & (files[k]["a_HWW_V2"] >= 0.99) for k in files}, 
    "SR1b" : {k:  (files[k]["nb_t_deep_in"] == 0 ) & (files[k]["MH_Reco"] >= 50) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] <= 0.25) & (files[k]["a_HWW_V2"] >= 0.92) & (files[k]["a_HWW_V2"] < 0.99) for k in files},   
    "SR2a" : {k:  (files[k]["nb_t_deep_in"] == 0 ) & (files[k]["MH_Reco"] >= 50) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] > 0.25) & (files[k]["a_HWW_V2"] >= 0.99) & (files[k]["DPhi"] < 0.8)for k in files}, 
    "SR2b" : {k:  (files[k]["nb_t_deep_in"] == 0 ) & (files[k]["MH_Reco"] >= 50) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] > 0.25) & (files[k]["a_HWW_V2"] >= 0.92) & (files[k]["a_HWW_V2"] < 0.99) & (files[k]["DPhi"] < 0.8)for k in files},   
    "CR1"  : {k:  (files[k]["nb_t_deep_in"] == 0 ) & (files[k]["MH_Reco"] >= 50) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] <= 0.25) & (files[k]["a_HWW_V2"] < 0.92) for k in files},
    "CR2"  : {k:  (files[k]["nb_t_deep_in"] == 0 ) & (files[k]["MH_Reco"] >= 50) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] > 0.25)  & (files[k]["a_HWW_V2"] < 0.92) & (files[k]["DPhi"] < 0.8) for k in files},   
    }

# CR: 0.6-0.8

In [10]:
CUT_BLINDED = {
        "SR1a_blinded" : {k: (files[k]["nb_t_deep_in"] == 0 ) & (files[k]["MH_Reco"] >= 50) & ((files[k]["MH_Reco"] <= 90) | (files[k]["MH_Reco"] >= 150)) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] <= 0.25) & (files[k]["a_HWW_V2"] >= 0.99) for k in files}, 
        "SR1b_blinded" : {k: (files[k]["nb_t_deep_in"] == 0 ) & (files[k]["MH_Reco"] >= 50) & ((files[k]["MH_Reco"] <= 90) | (files[k]["MH_Reco"] >= 150)) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] <= 0.25) & (files[k]["a_HWW_V2"] >= 0.92) & (files[k]["a_HWW_V2"] < 0.99) for k in files},   
        "SR2a_blinded" : {k: (files[k]["nb_t_deep_in"] == 0 ) & (files[k]["MH_Reco"] >= 50) & ((files[k]["MH_Reco"] <= 90) | (files[k]["MH_Reco"] >= 150)) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] > 0.25) & (files[k]["a_HWW_V2"] >= 0.99) & (files[k]["DPhi"] < 0.8)for k in files}, 
        "SR2b_blinded" : {k: (files[k]["nb_t_deep_in"] == 0 ) & (files[k]["MH_Reco"] >= 50) & ((files[k]["MH_Reco"] <= 90) | (files[k]["MH_Reco"] >= 150)) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] > 0.25) & (files[k]["a_HWW_V2"] >= 0.92) & (files[k]["a_HWW_V2"] < 0.99) & (files[k]["DPhi"] < 0.8)for k in files},   
        "CR1_blinded"  : {k: (files[k]["nb_t_deep_in"] == 0 ) & (files[k]["MH_Reco"] >= 50) & ((files[k]["MH_Reco"] <= 90) | (files[k]["MH_Reco"] >= 150)) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] <= 0.25) & (files[k]["a_HWW_V2"] < 0.92) for k in files},
        "CR2_blinded"  : {k: (files[k]["nb_t_deep_in"] == 0 ) & (files[k]["MH_Reco"] >= 50) & ((files[k]["MH_Reco"] <= 90) | (files[k]["MH_Reco"] >= 150)) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] > 0.25)  & (files[k]["a_HWW_V2"] < 0.92) & (files[k]["DPhi"] < 0.8)for k in files},   
}

### Save hist templates to pkl files

We note here that no particular operation is needed for QCD, since we only need raw QCD MC ratio as initial tranfer factor in the actual QCD prediction, and rhalphabet method will use (data - other bkg) in fail(control) region and perform simultaneous fit with pass(signal) region

In [11]:

def save_pkl(files, path_str = plot_dir, template_file = "hists_templates_run2.pkl"):
    regions = ["SR1a","SR1b","CR1","SR2a","SR2b","CR2"] #signal regions or control regions
    # samples = list(['data','QCD','Top','WJets','Rest','TotalSignal','ggF','VH','ttH','VBF']) 
    samples = list(['data','QCD','TT','ST','WJets','Rest','ggF','WH','ZH','ttH','VBF']) # ignore the combined "TotalSignal" for a test
    weights = [] #uncertainties to be added 
    templates = {}
    for region in regions:
        templates[region] = hist2.Hist(
            hist2.axis.StrCategory(samples, name="Sample"),
            # hist2.axis.StrCategory([], name="systematic", growth=True),
            *[shape_var.axis for shape_var in shape_vars],
            storage="weight",
            ) #initialize a hist object
        for sample in samples:
            data = files[sample][CUT[region][sample]] if ("blinded" not in region) else files[sample][CUT_BLINDED[region][sample]]
            templates[region].fill(
                Sample=sample,
                # systematic="nominal",
                MH_Reco=data["MH_Reco"],
                weight=data["weight"],
            )
            if sample == "data": 
                if (region.endswith("a") or region.endswith("b")):
                    # blind signal mass windows in pass region in data
                    print("blind data of ",region)
                    for i, shape_var in enumerate(shape_vars):
                        if shape_var.blind_window is not None:
                            blindBins(templates[region], shape_var.blind_window, "data", axis=i) 
            for weight in weights:
                pass #uncertainties to be added
        print("done fill template ",region)        
    #Creates blinded copies of each region's templates and saves a pickle of the templates
    blind_window = shape_vars[0].blind_window
    for label, template in list(templates.items()):
        blinded_template = deepcopy(template)
        blindBins(blinded_template, blind_window)
        templates[f"{label}Blinded"] = blinded_template
    with open(path_str + template_file, "wb") as fp:
        pkl.dump(templates, fp) # dump the templates of each region in a pkl file
        print("Saved templates to", template_file)

In [12]:
save_pkl(files = files)


blind data of  SR1a
done fill template  SR1a
blind data of  SR1b
done fill template  SR1b
done fill template  CR1


blind data of  SR2a
done fill template  SR2a
blind data of  SR2b
done fill template  SR2b
done fill template  CR2
Saved templates to hists_templates_run2.pkl


### some test about the output templates

In [13]:
with open("/home/pku/zhaoyz/Higgs/boostedHWW/postprocessing/templates/29Jan2024/hists_templates_run2.pkl","rb") as f:
    hists_template1 = pkl.load(f)
# hists_template["pass"]["QCD",:]  
# hists_template["pass"]["QCD",:].sum().value
# hists_template["CR2"]["QCD",:]
hists_template1["SR2a"]["data",:]

Hist(Regular(20, 50, 250, name='MH_Reco', label='Higgs candidate MET recovery mass [GeV]'), storage=Weight()) # Sum: WeightedSum(value=439, variance=439) (WeightedSum(value=478, variance=478) with flow)

In [14]:
hists_template1["SR1aBlinded"]

Hist(
  StrCategory(['data', 'QCD', 'Top', 'WJets', 'Rest', 'ggF', 'VH', 'ttH', 'VBF'], name='Sample'),
  Regular(20, 50, 250, name='MH_Reco', label='Higgs candidate MET recovery mass [GeV]'),
  storage=Weight()) # Sum: WeightedSum(value=15172.1, variance=49488.2) (WeightedSum(value=16346.9, variance=50955.8) with flow)

In [15]:
sample_template1 = hists_template1["SR1a"]["QCD", :]
err = sample_template1.variances()
err

array([4340.60013481, 4133.65798645, 6723.26535784, 5317.90207409,
       4784.74330639, 3802.64412873, 7207.47248974, 5552.02268741,
       4345.83959678, 4780.61467887, 4704.34121613, 3050.58159605,
       3283.55190705, 2823.03592685, 1516.18951594, 1880.52420034,
       2392.39906575,  943.09371738,  929.99007566,  667.823064  ])

In [16]:
for i , axis in enumerate(hists_template1["SR1a"].axes[1:]):
    print(i, axis)

0 Regular(20, 50, 250, name='MH_Reco', label='Higgs candidate MET recovery mass [GeV]')


### Some test of HHbbVV analysis for reference

In [26]:
# with open("/home/pku/zhaoyz/Higgs/HHbbVV/src/HHbbVV/postprocessing/templates/23Jun14/2018_templates.pkl","rb") as f:
with open("/ospool/cms-user/yuzhe/BoostedHWW/prediction/HHbbVV/src/HHbbVV/postprocessing/templates/24Mar15UpdateData/2018_templates.pkl","rb") as f:
    hists_template2 = pkl.load(f)
# hists_template["pass"]["QCD",:]  
# hists_template["pass"]["QCD",:].sum().value


In [27]:
hists_template2.keys()

dict_keys(['pass', 'fail', 'pass_JES_up', 'fail_JES_up', 'pass_JES_down', 'fail_JES_down', 'pass_JER_up', 'fail_JER_up', 'pass_JER_down', 'fail_JER_down', 'pass_JMS_up', 'fail_JMS_up', 'pass_JMS_down', 'fail_JMS_down', 'pass_JMR_up', 'fail_JMR_up', 'pass_JMR_down', 'fail_JMR_down', 'passBlinded', 'failBlinded', 'pass_JES_upBlinded', 'fail_JES_upBlinded', 'pass_JES_downBlinded', 'fail_JES_downBlinded', 'pass_JER_upBlinded', 'fail_JER_upBlinded', 'pass_JER_downBlinded', 'fail_JER_downBlinded', 'pass_JMS_upBlinded', 'fail_JMS_upBlinded', 'pass_JMS_downBlinded', 'fail_JMS_downBlinded', 'pass_JMR_upBlinded', 'fail_JMR_upBlinded', 'pass_JMR_downBlinded', 'fail_JMR_downBlinded'])

In [28]:
hists_template2["pass"]

Hist(
  StrCategory(['HHbbVV', 'ggHH_kl_2p45_kt_1_HHbbVV', 'ggHH_kl_5_kt_1_HHbbVV', 'ggHH_kl_0_kt_1_HHbbVV', 'VBFHHbbVV', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV', 'QCD', 'TT', 'ST', 'W+Jets', 'Z+Jets', 'Diboson', 'ggFHbb', 'VBFHbb', 'ZHbb', 'WHbb', 'ggZHbb', 'ttHbb', 'HWW', 'Data', 'HHbbVV_txbb_down', 'ggHH_kl_2p45_kt_1_HHbbVV_txbb_down', 'ggHH_kl_5_kt_1_HHbbVV_txbb_down', 'ggHH_kl_0_kt_1_HHbbVV_txbb_down', 'VBFHHbbVV_txbb_down', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_txbb_down', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_txbb_down', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_txbb_down', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_txbb_down', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_txbb_down', 'HHbbVV_pileup_down', 'ggHH_kl_2p45_kt_1_HHbbVV_pileup_down', 'ggHH_kl_5_kt_1_HHbbVV_pileup_down', 'ggHH_kl_0_kt_1_HHbbVV_pileup_down', 'VBFHHbbVV_pileup_down', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_pileup_down', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_pileup_d

In [35]:
hists_template2["pass"]["HHbbVV_pileup_up",:]

Hist(Regular(20, 50, 250, name='bbFatJetParticleNetMass', label='$m^{bb}_\\mathrm{Reg}$ (GeV)'), storage=Weight()) # Sum: WeightedSum(value=0.105892, variance=3.89652e-06)

In [33]:
hists_template2["pass"]["HHbbVV_pileup_up",:]

Hist(Regular(20, 50, 250, name='bbFatJetParticleNetMass', label='$m^{bb}_\\mathrm{Reg}$ (GeV)'), storage=Weight()) # Sum: WeightedSum(value=0.105892, variance=3.89652e-06)

In [36]:
hists_template2["pass"]["HHbbVV",:]

Hist(Regular(20, 50, 250, name='bbFatJetParticleNetMass', label='$m^{bb}_\\mathrm{Reg}$ (GeV)'), storage=Weight()) # Sum: WeightedSum(value=0.106171, variance=3.43708e-06)

In [19]:
hists_template2["passBlinded"]

Hist(
  StrCategory(['HHbbVV', 'ggHH_kl_2p45_kt_1_HHbbVV', 'ggHH_kl_5_kt_1_HHbbVV', 'ggHH_kl_0_kt_1_HHbbVV', 'VBFHHbbVV', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV', 'QCD', 'TT', 'ST', 'W+Jets', 'Z+Jets', 'Diboson', 'ggFHbb', 'VBFHbb', 'ZHbb', 'WHbb', 'ggZHbb', 'ttHbb', 'HWW', 'Data', 'HHbbVV_txbb_down', 'ggHH_kl_2p45_kt_1_HHbbVV_txbb_down', 'ggHH_kl_5_kt_1_HHbbVV_txbb_down', 'ggHH_kl_0_kt_1_HHbbVV_txbb_down', 'VBFHHbbVV_txbb_down', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_txbb_down', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_txbb_down', 'qqHH_CV_1_C2V_1_kl_2_HHbbVV_txbb_down', 'qqHH_CV_1_C2V_2_kl_1_HHbbVV_txbb_down', 'qqHH_CV_1_C2V_1_kl_0_HHbbVV_txbb_down', 'HHbbVV_pileup_down', 'ggHH_kl_2p45_kt_1_HHbbVV_pileup_down', 'ggHH_kl_5_kt_1_HHbbVV_pileup_down', 'ggHH_kl_0_kt_1_HHbbVV_pileup_down', 'VBFHHbbVV_pileup_down', 'qqHH_CV_1_C2V_0_kl_1_HHbbVV_pileup_down', 'qqHH_CV_1p5_C2V_1_kl_1_HHbbVV_pileup_d

### test how to load and use the *.pkl template file

In [22]:
def get_template(h, sample):
    ''' 
    histogram h Hist, with axes:["samples","systematic","MH_Reco"]
    sample is sample name in ["QCD",...,"data"]
    '''
    mass_axis = 1 #axis index
    massbins = h.axes[mass_axis].edges
    return (h[sample, :].values(), massbins, "MH_Reco")

a = get_template(hists_template1["SR3a"],"QCD")
a

KeyError: 'SR3a'

### Some other test

In [ ]:
regions = {
        "CR1" :{"SRa": "SR1a","SRb":"SR1b"},
        "CR2" :{"SRa": "SR2a","SRb":"SR2b"},
        "CR3" :{"SRa": "SR3a","SRb":"SR3b"},
        }

regions_blinded = { key_fail + "_blinded": {key_pass + "_blinded" : key_pass_ab + "_blinded" for key_pass , key_pass_ab in key_pass_dict.items()}  for key_fail , key_pass_dict in regions.items()}
regions_blinded.keys()

dict_keys(['CR1_blinded', 'CR2_blinded', 'CR3_blinded'])

In [ ]:
region = "SR1a_blinded"
pass_region = ("a_" in region)
pass_region

True

In [ ]:
region = "SR1aBlinded"
region_noblinded = region.split("Blinded")[0]
region_noblinded

'SR1a'

### function to scale the 